In [1]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

In [2]:
def leisure_pretreat(dframe) : 

    """
    
    Ver. 2
    
    여가 관련 통계조사 전처리 함수입니다.

    (Ver. 1 -> Ver. 2 : '평균' 컬럼 제외)

    """

    year_List = list(range(2016,2022))
    year_List = [value for value in year_List if value != 2017]   # 2017년도 빼기

    category = ["연령별", "가구소득별"]

    leis_sep = dframe.iloc[:,:2]

    conc_list_age = []
    conc_list_inc = []

    for yr in year_List : 

        pattern = str(yr)
        df_name = "DataFrame_" + str(yr)

        globals()[df_name] = dframe.filter(like=pattern)    # 조건에 만족하는 데이터 프레임 반환
        globals()[df_name] = pd.concat([leis_sep, globals()[df_name]], axis=1)     # 왼쪽 오른쪽으로 합치기, 인덱스가 다르면 outer join
        globals()[df_name] = globals()[df_name].replace("-",0)
        globals()[df_name].columns = globals()[df_name].iloc[0,:].to_list()
        globals()[df_name] = globals()[df_name].iloc[1:,:]
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(float)
        globals()[df_name].iloc[:,3:] = globals()[df_name].iloc[:,3:].apply(lambda x : x * globals()[df_name].iloc[:,2] / 100)  # 백분율로 표시된 값을 실제 값으로 변환
        globals()[df_name] = globals()[df_name].drop("표본수",axis=1)  # 합계 부분 삭제
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(int)

        if "평균" in globals()[df_name].columns.to_list() : 
            globals()[df_name].drop("평균", axis=1, inplace=True)
            
        for cat1 in category : 

            df_name_c1 = "DataFrame_" + str(yr) + cat1
            globals()[df_name_c1] = globals()[df_name][globals()[df_name]["통계분류(1)"]==cat1].iloc[:,1:]
            globals()[df_name_c1].insert(0, "연도별", yr)
            globals()[df_name_c1].rename({"통계분류(2)":cat1},axis=1,inplace=True)

            if cat1 == category[0] : 
                conc_list_age.append(globals()[df_name_c1])
            else : 
                conc_list_inc.append(globals()[df_name_c1])

    df1 = pd.concat(conc_list_age, axis=0)
    df2 = pd.concat(conc_list_inc, axis=0)

    df1.fillna(0, axis=1, inplace=True)
    df2.fillna(0, axis=1, inplace=True)

    return df1, df2

# 여가 관련 자료


## <font color="#FF5675"> 일과 여가 생활 균형 </font>

In [8]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')

In [9]:
A, B = leisure_pretreat(leisure)
A.head()

,연도별,연령별,일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
4,2016,15~19세,15,95,218,284,159,48,4
5,2016,20대,11,116,302,583,320,135,5
6,2016,30대,34,162,392,599,339,175,5
7,2016,40대,45,195,443,678,426,197,1
8,2016,50대,29,183,404,690,422,217,7


## <font color="#FFD73C"> 평일 여가 활동_관광 활동 </font>

In [5]:
trip = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_관광활동_20230510095154.csv", encoding="cp949")
# trip.info()
# trip.head()

In [6]:
trip_age, trip_inc = leisure_pretreat(trip)

In [7]:
trip_inc.tail()

,연도별,가구소득별,매우불만족한다,불만족 한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
34,2021,200~300만원,0,0,1,51,162,223,87
35,2021,300~400만원,0,0,3,59,207,304,130
36,2021,400~500만원,0,0,0,58,217,273,109
37,2021,500~600만원,0,0,0,57,191,236,94
38,2021,600만원 이상,0,0,0,57,216,348,199


## <font color="#FFD73C"> 평일 여가 활동_문화예술 관람 활동 </font>

In [14]:
artCult = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_문화예술_관람활동_20230519170721.csv", encoding="cp949")
artCult.head()

,통계분류(1),통계분류(2),2016,2016.1,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,...,2020.6,2020.7,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7
0,통계분류(1),통계분류(2),표본수,매우 불만족한다,불만족한다,다소 불만족한다,보통이다,다소 만족한다,만족한다,매우 만족한다,...,매우 만족한다,평균,표본수,불만족한다,다소 불만족한다,보통이다,다소 만족한다,만족한다,매우 만족한다,평균
1,전체,소계,3957,0,0.1,0.3,5.2,28.5,55.5,10.4,...,13.7,5.7,2068,0,0.2,5.6,30.8,47.4,15.9,5.7
2,성별,남성,1687,0,0.2,0.3,4.9,30.6,54.2,9.7,...,13.8,5.7,1011,-,0.4,6.1,30.1,46.5,16.9,5.7
3,성별,여성,2270,0,0.1,0.2,5.4,26.7,56.6,11,...,13.7,5.7,1057,0,-,5.1,31.5,48.3,15,5.7
4,연령별,15~19세,388,0,0,-,7,29.7,54,9.4,...,18.8,5.8,175,-,-,2.5,39.1,32.2,26.2,5.8


In [15]:
artCult_age, artCult_inc = leisure_pretreat(artCult)
print(artCult_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-*-")
print(artCult_inc.tail())


    연도별     연령별 매우 불만족한다 불만족한다 다소 불만족한다 보통이다 다소 만족한다 만족한다 매우 만족한다
4  2016  15~19세      0.0   0.0        0   27     115  209      36
5  2016     20대      0.0   3.0        1   27     260  543      93
6  2016     30대      0.0   0.0        2   42     231  449      83
7  2016     40대      0.0   1.0        3   46     249  419      79
8  2016     50대      0.0   0.0        1   26     156  315      63
-*-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우 불만족한다 불만족한다 다소 불만족한다 보통이다 다소 만족한다 만족한다 매우 만족한다
34  2021  200~300만원        0   0.0        2   12      77  125      44
35  2021  300~400만원        0   0.0        0   19      83  178      64
36  2021  400~500만원        0   0.0        0   17     130  181      35
37  2021  500~600만원        0   0.0        1   19     136  156      49
38  2021   600만원 이상        0   0.0        0   35     155  255     103


## <font color="#FFD73C"> 평일 여가 활동_사회 및 기타 </font>

In [16]:
sc_etc = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_사회_및_기타활동_20230510093021.csv", encoding="cp949")

In [17]:
sc_etc_age, sc_etc_inc = leisure_pretreat(sc_etc)
print(sc_etc_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(sc_etc_inc.tail())

    연도별     연령별 매우 불만족한다 불만족한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우 만족한다
4  2016  15~19세      0.0     2       2   42    201  364     130
5  2016     20대      1.0     0       5   66    374  689     228
6  2016     30대      1.0     1       3   97    474  749     219
7  2016     40대      0.0     1       7  111    578  865     227
8  2016     50대      0.0     0       1  103    495  861     293
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우 불만족한다 불만족한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우 만족한다
34  2021  200~300만원      0.0     0       7  157    390  573     193
35  2021  300~400만원      0.0     0       6  195    488  749     284
36  2021  400~500만원      1.0     0       2  137    457  667     199
37  2021  500~600만원      0.0     2       5  109    383  558     216
38  2021   600만원 이상      0.0     0       1  114    518  659     293


## <font color="#FFD73C"> 평일 여가 활동_스포츠 참여 활동 </font>

In [ ]:
doSports = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_스포츠_참여활동_20230510095111.csv", encoding="cp949")

In [ ]:
doSports_age, doSports_inc = leisure_pretreat(doSpots)
print(doSports_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(doSports_inc.tail())

    연도별     연령별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
4  2016  15~19세     0.0      0       2   32    142  231     60
5  2016     20대     0.0      0       3   49    244  467    138
6  2016     30대     0.0      0       3   47    275  485    164
7  2016     40대     1.0      2       9   56    309  497    180
8  2016     50대     0.0      0       2   27    233  433    166
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
34  2021  200~300만원     0.0      0       2   44    166  253    121
35  2021  300~400만원     0.0      0       0   63    209  335    159
36  2021  400~500만원     0.0      0       0   60    217  300    141
37  2021  500~600만원     1.0      0       2   49    181  317    130
38  2021   600만원 이상     0.0      0       2   40    220  376    254


## <font color="#FFD73C"> 평일 여가 활동_취미 오락 활동 </font>

In [21]:
games = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_취미・오락활동_20230510092915.csv", encoding="cp949")

In [22]:
games_age, games_inc = leisure_pretreat(games)
print(games_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(games_inc.tail())

    연도별     연령별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소 만족한다 만족한다 매우만족한다
4  2016  15~19세       0      0       1   42     237  370    148
5  2016     20대       0      1       1   74     453  698    208
6  2016     30대       1      0       6  112     530  806    178
7  2016     40대       0      0      11  123     556  959    214
8  2016     50대       1      0       7   77     563  948    211
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소 만족한다 만족한다 매우만족한다
34  2021  200~300만원       0      0       2  140     398  573    200
35  2021  300~400만원       0      0       5  164     512  819    252
36  2021  400~500만원       0      0       4  112     482  696    205
37  2021  500~600만원       0      1       1  119     395  544    233
38  2021   600만원 이상       0      0       0  137     499  677    338


## 일과 여가 생활 균형_연습버전

In [ ]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [ ]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [ ]:
leisure_T.replace("-",0,inplace=True)

In [ ]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

In [ ]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [ ]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [ ]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

In [ ]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [ ]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

In [ ]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [ ]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

In [ ]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


In [ ]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)